In [1]:
!kaggle competitions download -f train_jpg.zip --path ./data
!kaggle competitions download -f test_jpg.zip --path ./data
!unzip -qo ./data/train_jpg.zip
!unzip -qo ./data/test_jpg.zip

Using competition: avito-demand-prediction
 19%|███████▌                               | 9.52G/49.4G [01:11<04:35, 155MB/s]

100%|██████████████████████████████████████▉| 49.4G/49.4G [06:38<00:00, 136MB/s]


100%|███████████████████████████████████████| 49.4G/49.4G [06:38<00:00, 133MB/s]
Using competition: avito-demand-prediction
 31%|████████████                           | 5.79G/18.7G [00:47<01:43, 134MB/s]

100%|██████████████████████████████████████▉| 18.7G/18.7G [02:28<00:00, 115MB/s]
100%|███████████████████████████████████████| 18.7G/18.7G [02:28<00:00, 135MB/s]


In [1]:
from keras.preprocessing import image
from keras.applications.xception import Xception, preprocess_input
import glob
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
import dotenv

dotenv.load_dotenv('.env')

Using TensorFlow backend.


True

In [2]:
model = Xception(weights='imagenet', include_top=False, pooling='max')

In [3]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


In [0]:
types = ['train', 'test']

for t in types:
  paths = glob.glob('./data/competition_files/%s_jpg/*'%t)
  image_ids = []
  BATCH_SIZE = 50
  features = np.array([]).reshape(0, 2048)
  
  for path_batch in tqdm(batch(paths, BATCH_SIZE), total=len(paths) // BATCH_SIZE):
      ids = [os.path.splitext(os.path.basename(p))[0] for p in path_batch]
      imgs = np.asarray([image.img_to_array(image.load_img(p, target_size=(224, 224))) for p in path_batch])
      imgs = preprocess_input(imgs)
      pred = model.predict(imgs)

      image_ids += ids
      features = np.vstack([features, pred])
      
  df = pd.DataFrame(features, columns=['xception_max_%s'%i for i in range(2048)])
  df['image_id'] = image_ids
  df.columns = df.columns.astype(str)
  !mkdir image_features
  df.set_index('image_id').to_parquet('./image_features/%s_xception_max.snappy.parquet'%t)
  !gsutil rsync ./image_features gs://{os.environ['GCP_BUCKET']}/image_features

  9%|▉         | 2595/27816 [57:26<9:18:19,  1.33s/it]

In [23]:
pd.read_parquet('./image_features/train_xception.snappy.parquet')

,xception_0,xception_1,xception_2,xception_3,xception_4,xception_5,xception_6,xception_7,xception_8,xception_9,...,xception_2038,xception_2039,xception_2040,xception_2041,xception_2042,xception_2043,xception_2044,xception_2045,xception_2046,xception_2047
image_id,,,,,,,,,,,,,,,,,,,,,
700344a773b1bfb5d9eba821623103425aa72e1976ba71b0e393c22e1b3b2ff1,0.001728,0.127272,0.000000,0.418413,0.000000,0.459921,0.000000,0.004271,0.000000,0.000000,...,0.000000,0.000000,0.078988,0.050702,0.177119,0.311513,0.638233,0.365060,0.012108,0.000000
8492d3292f25c55124ec6905e77297893f5ddbce55d9140bfdd8c3f5760bb0b1,0.274815,0.050180,0.278069,0.338182,0.600384,0.048850,0.111873,0.054596,0.000000,0.123587,...,0.000000,0.239241,0.578349,0.000000,0.001406,0.000000,0.072845,0.000000,0.031958,0.025320
1d03f5fbcc4632d6e34fce17d259a171ddd989201ce5d5a94d98c4f84bcaf7ec,0.890300,0.000000,0.040222,2.266773,0.413060,0.254859,0.000000,0.097757,0.010556,0.001422,...,0.000000,0.157796,0.000000,0.153983,0.118170,0.146818,0.115142,0.855579,0.000000,0.052531
8d04fbb466f02bad8554dc046165350b7be46c9928877685c420380c2335ec3e,0.004507,0.065810,0.000000,0.014132,0.151821,0.544012,0.724705,0.827198,0.152903,0.038421,...,0.003414,0.000000,0.013162,0.022087,0.486857,0.036988,0.042709,0.117756,0.007158,0.013264
4811ebbc338265bca641f9c0197dbe546758489dd5525527a92e99b500b5358a,0.012674,0.000000,0.000000,0.647734,0.346693,0.509855,0.253654,0.969924,0.213951,0.638029,...,0.000000,0.021779,0.197228,0.325992,0.368942,0.182379,0.185252,0.000000,0.000255,0.220475
64bf334d0a3777d3cb509a0582d0c6f45b7c16f3360bc15c191139c2838075ac,0.320716,0.076119,0.039124,1.346846,0.551578,0.234990,0.049856,0.201968,0.000000,0.000000,...,0.109320,0.001176,0.021169,0.622850,0.005325,0.001699,0.033415,0.165559,0.000000,0.017610
0045d2214be9fb4bd6f9301ef8840861489d5a48d083dda15814fef50bf15955,0.380018,0.373736,0.175915,0.215934,0.059736,0.090436,0.047381,0.581269,0.246122,0.073527,...,0.000000,0.042732,0.031271,0.344092,0.003255,0.264058,0.582552,0.032897,0.060325,0.347846
61cd3ead0f74004edb085203f47c23b675107784acd2326ec0f80cb352249160,0.626636,0.238539,0.426280,0.918939,0.078224,0.072037,0.559903,0.717444,0.385229,0.384050,...,0.326102,0.255814,0.400558,0.279018,0.248877,0.816545,0.120231,0.127704,0.420635,0.297233
3a88f3b9784c068f02d1511ae6cbd2a02b23bdbcb7a3ab91424e3dfbc638a136,0.726937,0.000994,0.004564,0.797198,0.386366,0.031619,0.006682,0.000000,0.477940,0.274443,...,0.063586,0.000000,0.000000,0.065586,0.002825,0.000000,0.000000,0.000775,0.045787,0.000000
